In [1]:
!pip install PyMuPDF
!pip install rank_bm25
!pip install langchain
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.1/269.1 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 20.8 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.1/163.1 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [32]:
import numpy as np
import fitz  # PyMuPDF
import nltk
import string
import sklearn
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from rank_bm25 import BM25Okapi
import json

In [33]:
# NLTK datasets
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
def load_dataset(path):
    with open(path) as file:
        dataset = json.load(file)
    return dataset

dataset = load_dataset('/content/drive/MyDrive/Data/train-v2.0.json')

In [36]:
def extract_text_from_pdf(pdf_path):
  doc = fitz.open(pdf_path)
  texts = []
  for page in doc:
      texts.append(page.get_text())
  return texts

In [35]:
document_context = '/content/drive/MyDrive/Data/contexts_train.pdf'
docs = extract_text_from_pdf(document_context)

In [11]:
# Preprocessing
def preprocess_documents(docs):
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))
    preprocessed_docs = []
    for doc in docs:
        doc = doc.lower()
        doc = doc.translate(str.maketrans('', '', string.punctuation))
        tokens = word_tokenize(doc)
        tokens = [lemmatizer.lemmatize(stemmer.stem(word)) for word in tokens if word not in stop_words]
        preprocessed_docs.append(' '.join(tokens))
    return preprocessed_docs

In [37]:
preprocessed_docs = preprocess_documents(docs)

In [38]:
# Vectorizing the preprocessed documents
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(docs)

In [21]:
# Outputs
print("Preprocessed Documents:")
num_docs_to_print = 10

for i in range(num_docs_to_print):
  print(docs[i][:100])

print("\nShape of the TF-IDF matrix:", X.shape)

Preprocessed Documents:
Beyoncé Giselle Knowles-Carter (/biIIjInseI/ bee-YON-say) (born September 4, 1981) is an
American si
performed was not good. In 1995 Beyoncé's father resigned from his job to manage the group. The
move
album, with Missy Elliott, MC Lyte, and Free which was also used to promote the film. Another of
Bey
Achieving the accomplishment of becoming her longest-running Hot 100 single in her career, "Halo"'s

"End of Time". "Eat, Play, Love", a cover story written by Beyoncé for Essence that detailed her 201
America festival in early September and also the Global Citizen Festival later that month. Beyoncé
m
In 2015 Beyoncé signed an open letter which the ONE Campaign had been collecting signatures for;
the
sharing them with producers.
In 2001, she became the first African-American woman and second woman s
Described as being "sexy, seductive and provocative" when performing on stage, Beyoncé has said
that
singles – particularly R&B; singles – regained their status as pop'

In [22]:
# Splitting Text into Chunks i.e. each document into sentences
docs_sentences = [sent_tokenize(doc) for doc in docs]

In [24]:
# Generating Embeddings
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

# Flattening the list of sentences
sentences = [sentence for doc in docs_sentences for sentence in doc]
sentence_embeddings = model.encode(sentences)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# **BM25 Model**

In [25]:
# BM25 for Sparse Retrieval
from rank_bm25 import BM25Okapi

# Tokenizing the sentences for BM25
tokenized_docs = [doc.split() for doc in sentences]
bm25 = BM25Okapi(tokenized_docs)

In [39]:
tfidf_matrix = vectorizer.fit_transform(docs)

In [40]:
# Query and Retrieve Documents
query = "In what country is Normandy located?"
tokenized_query = query.split()

# Getting scores for each document or sentence
doc_scores = bm25.get_scores(tokenized_query)

# Finding the highest scoring documents
top_doc_indices = np.argsort(doc_scores)[::-1][:5]

for idx in top_doc_indices:
    print(f"Doc {idx + 1}: {sentences[idx]} (Score: {doc_scores[idx]:.2f})")
    print("\n")

Doc 89892: The country joined what is now the European Union in
1981. (Score: 14.99)


Doc 27494: In 1214 John began his final campaign to reclaim Normandy from Philip. (Score: 13.57)


Doc 38066: Civil disorder spread
throughout the country in what became known as the Autumn uprising. (Score: 12.08)


Doc 44383: In English, the country is popularly known as either "Burma" or "Myanmar" i/ImjIInImIIr/. (Score: 11.14)


Doc 30985: In the Channel Islands and Isle of Man, which are Crown dependencies rather than
separate realms, she is known as Duke of Normandy and Lord of Mann, respectively. (Score: 11.12)




# **TF-IDF MODEL**

In [45]:
docs_for_tfidf = sentences
tfidf_matrix = vectorizer.fit_transform(docs_for_tfidf)

In [41]:
def tfidf_retrieve(query):
    # Transform the query to the same TFIDF representation
    query_tfidf = vectorizer.transform([query])

    # Calculate cosine similarities between the query and all documents
    cosine_similarities = linear_kernel(query_tfidf, tfidf_matrix).flatten()

    # Get indices of the documents sorted by similarity (descending)
    related_docs_indices = cosine_similarities.argsort()[:-6:-1]

    return related_docs_indices, cosine_similarities

In [46]:
query = "In what country is Normandy located?"
top_doc_indices, scores = tfidf_retrieve(query)

for idx in top_doc_indices:
    print(f"Doc {idx + 1}: {sentences[idx]} (Score: {scores[idx]:.2f})")
    print("\n")

Doc 18948: Besides the settlements in Ireland, England, and Normandy, further settlement took
place in what became Russia and in Iceland. (Score: 0.33)


Doc 39899: It is located in the Valley of (Score: 0.32)


Doc 89892: The country joined what is now the European Union in
1981. (Score: 0.32)


Doc 9689: They are typically
located in the country or along a highway. (Score: 0.32)


Doc 61977: What is
the difference?" (Score: 0.30)


